## What this notebook is

This is a inference notebook using 4-bit quantized [Gemma-2 9b Instruct](https://blog.google/technology/developers/google-gemma-2/) and a LoRA adapter trained using the script I uploaded [here](https://www.kaggle.com/code/emiz6413/gemma-2-9b-4-bit-qlora-finetune).
Although we can choose to merge the LoRA adapter to the base model for faster inference, naively doing so could introduce non-negligible quantization error. Therefore, I opted to keep the LoRA adapter unmerged. 

## Result

| subset | log loss |
| - | - |
| eval set | 0.9371 |
| public LB | 0.941 |

The submission takes around 4 hours with `max_length=2048` without TTA.

In [1]:
# !pip install transformers peft accelerate bitsandbytes \
#     -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [1]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import glob
import re
from tqdm import tqdm

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Qwen2ForSequenceClassification, Qwen2TokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

In [2]:
# assert torch.cuda.device_count() == 2

## Configurations

In [3]:
@dataclass
class Config:
    gemma_dir = '../Qwen2.5-7B-Instruct/'
    lora_dir = './output/checkpoint-2400/'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()

# Load & pre-process Data 

In [4]:
# 读取 Markdown 文件
def read_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# 按句子分割并生成 chunk
def split_markdown_to_chunks(markdown_text):
    # 使用正则表达式匹配句子（以句号、问号、感叹号等结尾）
    sentences = re.split(r'(?<=[.!?])\s+', markdown_text.strip())
    
    # 每 3-5 句话分成一个 chunk
    chunk_size = 1  # 你可以调整每个 chunk 的大小
    chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]
    
    # 将句子列表重新组合为字符串（每个 chunk 为一个字符串）
    chunk_strings = [' '.join(chunk) for chunk in chunks]
    
    return chunk_strings

In [5]:
module_content_list = []
source_list = []
module_list = []
material_id_list = []

for row in pd.read_json("../data/测试 A 集/data.jsonl", lines=True).iterrows():
    module_name = row[1].rule.replace("该产品的", "").replace("在各材料中的定义没有冲突", "")
    
    print(f'module_name: {module_name}')


    # 提取每个材料中与module_name 相关的文本
    for path in glob.glob(f"../data/测试 A 集/materials/{row[1].material_id}/*/*"):
        markdown_text = read_markdown(path)
        source = path.split('/')[-2]
        chunks = split_markdown_to_chunks(markdown_text)
        
        module_content_list.append(chunks)
        source_list.append(source)
        module_list.append(module_name)
        material_id_list.append(row[1].material_id)

module_name: 基础产品销售信息
module_name: 赔付 & 领取规则
module_name: 责任免除
module_name: 投保条款
module_name: 投保条款
module_name: 责任免除
module_name: 续保条款
module_name: 责任免除
module_name: 出险条款
module_name: 保障相关时间
module_name: 赔付 & 领取规则
module_name: 责任免除
module_name: 责任免除
module_name: 保障责任
module_name: 退保条款
module_name: 保障责任
module_name: 赔付 & 领取规则
module_name: 退保条款
module_name: 投保条款
module_name: 续保条款
module_name: 出险条款
module_name: 投保条款
module_name: 续保条款
module_name: 责任免除
module_name: 与保障相关的时间
module_name: 保障责任
module_name: 责任免除
module_name: 保障责任
module_name: 责任免除
module_name: 保障责任
module_name: 责任免除
module_name: 术语解释
module_name: 与保障相关的时间
module_name: 基础产品销售信息
module_name: 责任免除
module_name: 责任免除
module_name: 赔付 & 领取规则
module_name: 基础产品销售信息
module_name: 赔付 & 领取规则
module_name: 附加条款
module_name: 退保条款
module_name: 与保障相关的时间
module_name: 续保条款
module_name: 与保障相关的时间
module_name: 保障责任
module_name: 投保条款
module_name: 附加条款
module_name: 保障责任
module_name: 出险条款
module_name: 投保条款
module_name: 术语解释
module_name: 赔付 & 领取规则
modu

In [8]:
test_data_df = pd.DataFrame({'material_id': material_id_list,
                             'module': module_list,
                            'source': source_list,
                            'content':module_content_list})
test_data_df['content'] = test_data_df['content'].apply(lambda x: [t.replace('\n', '').replace('\t', ' ') for t in x])
test_data_df.head()

,material_id,module,source,content
0,m_00001a,基础产品销售信息,INSURE_NOTICE,[投保须知本产品名称为“畅玩境内旅行保险”，由众安在线财产保险股份有限公司承保（以下简称“众...
1,m_00001a,基础产品销售信息,INTRODUCE_IMG,[畅玩境内旅行保险畅游中华·众安随行神奇敦煌帝都北京古都西安诗画西湖山水桂林浪漫三亚四重保障...
2,m_00001a,基础产品销售信息,HEAD_IMG,[null]
3,m_00001a,基础产品销售信息,CLAUSE,[众安在线财产保险股份有限公司 附加旅行银行卡盗刷保险条款 （众安备-家财【2015】附20...
4,m_00001a,基础产品销售信息,CLAUSE,[众安在线财产保险股份有限公司旅行意外伤害保险条款（互联网）注册号：C00017932312...


In [9]:
test_data_df = test_data_df.explode('content')
test_data_df

,material_id,module,source,content
0,m_00001a,基础产品销售信息,INSURE_NOTICE,投保须知本产品名称为“畅玩境内旅行保险”，由众安在线财产保险股份有限公司承保（以下简称“众安...
1,m_00001a,基础产品销售信息,INTRODUCE_IMG,畅玩境内旅行保险畅游中华·众安随行神奇敦煌帝都北京古都西安诗画西湖山水桂林浪漫三亚四重保障护...
2,m_00001a,基础产品销售信息,HEAD_IMG,null
3,m_00001a,基础产品销售信息,CLAUSE,众安在线财产保险股份有限公司 附加旅行银行卡盗刷保险条款 （众安备-家财【2015】附203...
3,m_00001a,基础产品销售信息,CLAUSE,2 保险财产 本附加险合同 中保险财产 指 被保险人名下的 银行卡（包括 储蓄卡...
...,...,...,...,...
1779,m_00066a,赔付 & 领取规则,INSURE_NOTICE,【投保须知】 一、基本信息 本产品为平安普惠医疗补充保 2024(互联网版)，所涉条款为《平...
1780,m_00066a,赔付 & 领取规则,ADDITIONAL_AGREEMENT,内容填写表（放在第一个sheet） 层级1 层级2 层级3 1、保险期间内，本保险合同每个被...
1781,m_00066a,赔付 & 领取规则,CLAUSE,中国平安财产保险股份有限公司平安产险补充医疗费用保险（普惠型）（互联网版）条款注册号：C00...
1781,m_00066a,赔付 & 领取规则,CLAUSE,当地基本医保经办部门出具的医疗费用结算证明原件、诊断证明、病历、医疗费用发票（包含费月清单）...


# Tokenize

In [10]:
def tokenize(
    tokenizer, module, text, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = [f"<prompt>: 文档中内容是否涉及到{m}？" for m in module]
    insurance_text = ["\n\n保险文本: " + t for t in text]

    text = [p + t for p, t in zip(prompt, insurance_text)]
    tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
    input_ids = tokenized.input_ids
    attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

In [11]:
%%time

tokenizer = Qwen2TokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token  # 很多中文模型没 pad_token，可复用 eos

data = pd.DataFrame()
data["material_id"] = test_data_df["material_id"]
data["source"] = test_data_df["source"]
data["module"] = test_data_df["module"]
data["content"] = test_data_df["content"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test_data_df["module"], test_data_df["content"])
data["length"] = data["input_ids"].apply(len)

CPU times: user 17.5 s, sys: 793 ms, total: 18.3 s
Wall time: 2.59 s


# Load model

In [12]:
# Load base model on GPU 0
device_0 = torch.device('cuda:0')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # or 'fp4'
    bnb_4bit_compute_dtype=torch.float16,
)


model_0 = Qwen2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    quantization_config=bnb_config,
    device_map=device_0,
    use_cache=False,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at ../Qwen2.5-7B-Instruct/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Load LoRA adapter

In [13]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)

In [14]:
model_0.config.pad_token_id = tokenizer.pad_token_id  # 使用 tokenizer 的 pad_token_id

# Inference


In [15]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    res_prob = []
    
    for start_idx in tqdm(range(0, len(df), batch_size)):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        res_prob.extend(proba[:, 1].tolist())
    
    df["is_contains"] = res_prob
    
    return df

In [16]:
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same

results = inference(data, model_0, device_0)

# proba = result_df["is_contains"].values

# print(f"elapsed time: {time.time() - st}")

100%|██████████| 2675/2675 [07:28<00:00,  5.96it/s]


In [17]:
results_copy = results[['material_id', 'source', 'module', 'content', 'is_contains']].copy()

In [18]:
results_copy.head()

,material_id,source,module,content,is_contains
4,m_00001a,CLAUSE,基础产品销售信息,相 关 关 系 22 附 录 B 23 参考 文 献 26 ...,3.212095e-07
1781,m_00066a,CLAUSE,赔付 & 领取规则,中国平安财产保险股份有限公司平安产险补充医疗费用保险（普惠型）（互联网版）条款注册号：C00...,4.323452e-05
4,m_00001a,CLAUSE,基础产品销售信息,众安在线财产保险股份有限公司旅行意外伤害保险条款（互联网）注册号：C000179323120...,1.032815e-02
1146,m_00067a,INSURE_NOTICE,退保条款,手 机下载我公司“ 人 保寿险管家APP”，注册后在“我的保单”查询保险合同信息。请您核对保...,7.029574e-07
0,m_00001a,INSURE_NOTICE,基础产品销售信息,投保须知本产品名称为“畅玩境内旅行保险”，由众安在线财产保险股份有限公司承保（以下简称“众安...,2.032284e-04


In [19]:
results_copy

,material_id,source,module,content,is_contains
4,m_00001a,CLAUSE,基础产品销售信息,相 关 关 系 22 附 录 B 23 参考 文 献 26 ...,3.212095e-07
1781,m_00066a,CLAUSE,赔付 & 领取规则,中国平安财产保险股份有限公司平安产险补充医疗费用保险（普惠型）（互联网版）条款注册号：C00...,4.323452e-05
4,m_00001a,CLAUSE,基础产品销售信息,众安在线财产保险股份有限公司旅行意外伤害保险条款（互联网）注册号：C000179323120...,1.032815e-02
1146,m_00067a,INSURE_NOTICE,退保条款,手 机下载我公司“ 人 保寿险管家APP”，注册后在“我的保单”查询保险合同信息。请您核对保...,7.029574e-07
0,m_00001a,INSURE_NOTICE,基础产品销售信息,投保须知本产品名称为“畅玩境内旅行保险”，由众安在线财产保险股份有限公司承保（以下简称“众安...,2.032284e-04
...,...,...,...,...,...
1667,m_00132a,HEAD_IMG,保障责任,null,9.960508e-01
1349,m_00112a,HEAD_IMG,投保条款,null,8.834373e-01
1341,m_00004a,HEAD_IMG,责任免除,null,8.976953e-01
98,m_00012a,HEAD_IMG,责任免除,null,8.976953e-01


In [21]:
results_copy = results_copy[results_copy['is_contains'] > 0.9][['material_id', 'source', 'module', 'content']]
results_copy

,material_id,source,module,content
203,m_00021a,CLAUSE,出险条款,原位癌，癌前病变，非浸润性癌，非侵袭性癌，肿瘤细胞未侵 犯基底层，上皮内瘤变，细胞不典型性...
988,m_00089a,CLAUSE,附加条款,所能提供的与确认保险事故的性质、原因等有关的其他证明和资料。如果委托他⼈代为申请，除上述证明...
1000,m_00017a,INSURE_NOTICE,与保障相关的时间,电子保单发出之日视为投保人的签收日，保险合同生效后，自签收日次日起算犹豫期，投保人享有 15...
960,m_00086a,CLAUSE,赔付 & 领取规则,1众安在线财产保险股份有限公司附加恶性肿瘤质子重离子医疗保险条款（互联网2022版A款）注册...
961,m_00086a,CLAUSE,赔付 & 领取规则,1众安在线财产保险股份有限公司附加恶性肿瘤——重度院外特定药品费用医疗保险条款（互联网 20...
...,...,...,...,...
1618,m_00032a,CLAUSE,保障责任,1.
265,m_00020a,INSURE_NOTICE,保障责任,5.
265,m_00020a,INSURE_NOTICE,保障责任,6.
265,m_00020a,INSURE_NOTICE,保障责任,6.


In [23]:
results_copy = results_copy[results_copy['content'] != 'null']
results_copy = results_copy[results_copy['content'].str.len() > 10]
results_copy

,material_id,source,module,content
203,m_00021a,CLAUSE,出险条款,原位癌，癌前病变，非浸润性癌，非侵袭性癌，肿瘤细胞未侵 犯基底层，上皮内瘤变，细胞不典型性...
988,m_00089a,CLAUSE,附加条款,所能提供的与确认保险事故的性质、原因等有关的其他证明和资料。如果委托他⼈代为申请，除上述证明...
1000,m_00017a,INSURE_NOTICE,与保障相关的时间,电子保单发出之日视为投保人的签收日，保险合同生效后，自签收日次日起算犹豫期，投保人享有 15...
960,m_00086a,CLAUSE,赔付 & 领取规则,1众安在线财产保险股份有限公司附加恶性肿瘤质子重离子医疗保险条款（互联网2022版A款）注册...
961,m_00086a,CLAUSE,赔付 & 领取规则,1众安在线财产保险股份有限公司附加恶性肿瘤——重度院外特定药品费用医疗保险条款（互联网 20...
...,...,...,...,...
488,m_00017a,CLAUSE,术语解释,术语的解释 ...........................................
1096,m_00097a,INTRODUCE_IMG,附加条款,为什么需要意外补充包?
1616,m_00032a,INTRODUCE_IMG,保障责任,为什么需要意外补充包?
817,m_00074a,CLAUSE,附加条款,争议处理 ............................................


In [24]:
res_df = results_copy.groupby(['material_id','source', 'module'], as_index=False).agg(list)
res_df

,material_id,source,module,content
0,m_00001a,CLAUSE,基础产品销售信息,[众安在线财产保险股份有限公司 附加旅行意外医疗费用补偿保险条款（互联网） 注册号：C000...
1,m_00001a,INTRODUCE_IMG,基础产品销售信息,[畅玩境内旅行保险畅游中华·众安随行神奇敦煌帝都北京古都西安诗画西湖山水桂林浪漫三亚四重保障...
2,m_00002a,CLAUSE,赔付 & 领取规则,"[住院天数 指被保险人在医院实际的住院治疗日数。住院满 24 小时为一日。 12., 3.1..."
3,m_00002a,INSURE_NOTICE,赔付 & 领取规则,[偿付能力：公司最近季度的综合偿付能力充足率、风险综合评级符合监管要求，详情请通过链接（ht...
4,m_00003a,ADDITIONAL_AGREEMENT,责任免除,[内容填写表（放在第一个sheet） 层级1 层级2 层级3 一、特别约定 1．本产品的等...
...,...,...,...,...
386,m_00137a,INSURE_NOTICE,附加条款,[1百万安心疗（2022 升级版，互联网专属）产品投保须知【一、产品责任/保障方案介绍】本产...
387,m_00138a,CLAUSE,基础产品销售信息,[所能提供的与确认保险事故的性质、原因等有关的其他证明和资料。如果委托他人代为申请，除上述证...
388,m_00138a,PRODUCT_MANUAL,基础产品销售信息,[核爆炸、核辐射或核污染。发生上述第 1 项情形导致被保险人身故的，合同终止，您已交足 2 ...
389,m_00139a,CLAUSE,基础产品销售信息,[释义⼀、分红保险⼆、周岁三、保单年度四、保险费约定⽀付⽇五、保单年度数六、减保七、现⾦价值...


In [25]:
final_res = []
grouped = res_df.groupby(['material_id', 'module'])

for (col1_val, col2_val), group in grouped:
    data_items = group[['source', 'content']].to_dict(orient='records')
    final_res.append({
        'material_id': col1_val,
        'module': col2_val,
        'matched_sentences': data_items
    })

In [26]:
# 转换为 DataFrame
final_res_df = pd.DataFrame(final_res)

# 显示结果
final_res_df

,material_id,module,matched_sentences
0,m_00001a,基础产品销售信息,"[{'source': 'CLAUSE', 'content': ['众安在线财产保险股份有..."
1,m_00002a,赔付 & 领取规则,"[{'source': 'CLAUSE', 'content': ['住院天数 指被保险人在..."
2,m_00003a,责任免除,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
3,m_00004a,投保条款,"[{'source': 'INSURE_NOTICE', 'content': ['保单及发..."
4,m_00004a,责任免除,"[{'source': 'CLAUSE', 'content': ['众安在线财产保险股份有..."
...,...,...,...
162,m_00135a,附加条款,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
163,m_00136a,附加条款,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
164,m_00137a,附加条款,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
165,m_00138a,基础产品销售信息,"[{'source': 'CLAUSE', 'content': ['所能提供的与确认保险事..."


In [27]:
rule_info = pd.read_json("../data/测试 A 集/data.jsonl", lines=True)
rule_info

,material_id,rule_id,rule
0,m_00001a,r_00001,该产品的基础产品销售信息在各材料中的定义没有冲突
1,m_00002a,r_00002,该产品的赔付 & 领取规则在各材料中的定义没有冲突
2,m_00003a,r_00003,该产品的责任免除在各材料中的定义没有冲突
3,m_00004a,r_00004,该产品的投保条款在各材料中的定义没有冲突
4,m_00005a,r_00005,该产品的投保条款在各材料中的定义没有冲突
...,...,...,...
195,m_00138a,r_00196,该产品的基础产品销售信息在各材料中的定义没有冲突
196,m_00120a,r_00197,该产品的附加条款在各材料中的定义没有冲突
197,m_00139a,r_00198,该产品的基础产品销售信息在各材料中的定义没有冲突
198,m_00061a,r_00199,该产品的责任免除在各材料中的定义没有冲突


In [28]:
matches = []

# 遍历 df_B 的每一行
for _, row_b in rule_info.iterrows():
    text = row_b['rule'].lower()
    b_material_id = row_b['material_id']
    
    # 遍历 df_A，查找 keyword 是否在 text 中出现
    for _, row_a in final_res_df.iterrows():
        keyword = row_a['module'].lower()
        if (keyword in text) and (b_material_id == row_a['material_id']):
            matches.append({
                'material_id': row_b['material_id'],
                'rule_id': row_b['rule_id'],
                'rule': row_b['rule'],
                'matched_sentences': row_a['matched_sentences']
            })

# 构建结果 DataFrame
df_matched = pd.DataFrame(matches)
df_matched

,material_id,rule_id,rule,matched_sentences
0,m_00001a,r_00001,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['众安在线财产保险股份有..."
1,m_00002a,r_00002,该产品的赔付 & 领取规则在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['住院天数 指被保险人在..."
2,m_00003a,r_00003,该产品的责任免除在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
3,m_00004a,r_00004,该产品的投保条款在各材料中的定义没有冲突,"[{'source': 'INSURE_NOTICE', 'content': ['保单及发..."
4,m_00005a,r_00005,该产品的投保条款在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['投保⼈、被保险⼈义务9..."
...,...,...,...,...
162,m_00138a,r_00196,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['所能提供的与确认保险事..."
163,m_00120a,r_00197,该产品的附加条款在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
164,m_00139a,r_00198,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['释义⼀、分红保险⼆、周..."
165,m_00061a,r_00199,该产品的责任免除在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."


In [29]:
submit_753 = pd.read_json("../data/submit_74.jsonl", lines=True)
submit_753

,material_id,rule_id,result
0,m_00001a,r_00001,False
1,m_00002a,r_00002,False
2,m_00003a,r_00003,False
3,m_00004a,r_00004,True
4,m_00005a,r_00005,False
...,...,...,...
195,m_00138a,r_00196,False
196,m_00120a,r_00197,False
197,m_00139a,r_00198,False
198,m_00061a,r_00199,False


In [30]:
submit_cur_df = pd.merge(submit_753, df_matched[['material_id', 'rule_id', 'rule']],how='left', on=['material_id', 'rule_id'])

In [31]:
df_matched[['material_id', 'rule_id', 'rule', 'matched_sentences']].to_csv('finetune_extra_module.csv', index=False)

In [32]:
submit_cur_df.loc[submit_cur_df['rule'].isna(), 'result'] = True

In [33]:
submit_cur_df

,material_id,rule_id,result,rule
0,m_00001a,r_00001,False,该产品的基础产品销售信息在各材料中的定义没有冲突
1,m_00002a,r_00002,False,该产品的赔付 & 领取规则在各材料中的定义没有冲突
2,m_00003a,r_00003,False,该产品的责任免除在各材料中的定义没有冲突
3,m_00004a,r_00004,True,该产品的投保条款在各材料中的定义没有冲突
4,m_00005a,r_00005,False,该产品的投保条款在各材料中的定义没有冲突
...,...,...,...,...
195,m_00138a,r_00196,False,该产品的基础产品销售信息在各材料中的定义没有冲突
196,m_00120a,r_00197,False,该产品的附加条款在各材料中的定义没有冲突
197,m_00139a,r_00198,False,该产品的基础产品销售信息在各材料中的定义没有冲突
198,m_00061a,r_00199,False,该产品的责任免除在各材料中的定义没有冲突


In [34]:
submit_753[submit_753['result'] == True]

,material_id,rule_id,result
3,m_00004a,r_00004,True
8,m_00009a,r_00009,True
9,m_00010a,r_00010,True
10,m_00011a,r_00011,True
11,m_00012a,r_00012,True
...,...,...,...
183,m_00041a,r_00184,True
184,m_00088a,r_00185,True
185,m_00131a,r_00186,True
190,m_00134a,r_00191,True


In [35]:
submit_cur_df[submit_cur_df['result'] == True]

,material_id,rule_id,result,rule
3,m_00004a,r_00004,True,该产品的投保条款在各材料中的定义没有冲突
8,m_00009a,r_00009,True,该产品的出险条款在各材料中的定义没有冲突
9,m_00010a,r_00010,True,NaN
10,m_00011a,r_00011,True,该产品的赔付 & 领取规则在各材料中的定义没有冲突
11,m_00012a,r_00012,True,该产品的责任免除在各材料中的定义没有冲突
...,...,...,...,...
184,m_00088a,r_00185,True,该产品的保障责任在各材料中的定义没有冲突
185,m_00131a,r_00186,True,该产品的出险条款在各材料中的定义没有冲突
187,m_00129a,r_00188,True,NaN
190,m_00134a,r_00191,True,NaN


In [38]:
df_matched

,material_id,rule_id,rule,matched_sentences
0,m_00001a,r_00001,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['众安在线财产保险股份有..."
1,m_00002a,r_00002,该产品的赔付 & 领取规则在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['住院天数 指被保险人在..."
2,m_00003a,r_00003,该产品的责任免除在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
3,m_00004a,r_00004,该产品的投保条款在各材料中的定义没有冲突,"[{'source': 'INSURE_NOTICE', 'content': ['保单及发..."
4,m_00005a,r_00005,该产品的投保条款在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['投保⼈、被保险⼈义务9..."
...,...,...,...,...
162,m_00138a,r_00196,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['所能提供的与确认保险事..."
163,m_00120a,r_00197,该产品的附加条款在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."
164,m_00139a,r_00198,该产品的基础产品销售信息在各材料中的定义没有冲突,"[{'source': 'CLAUSE', 'content': ['释义⼀、分红保险⼆、周..."
165,m_00061a,r_00199,该产品的责任免除在各材料中的定义没有冲突,"[{'source': 'ADDITIONAL_AGREEMENT', 'content':..."


In [39]:
# 转换函数：将 content 列表合并成字符串
def normalize_content(entry_list):
    res_list = []
    for item in entry_list:
        if isinstance(item.get('content'), list):
             res_list.append({f"{item['source']}": '\n'.join(item['content'])})
    return res_list


In [40]:
df_matched['matched_sentences'] = df_matched['matched_sentences'].apply(normalize_content)
df_matched

,material_id,rule_id,rule,matched_sentences
0,m_00001a,r_00001,该产品的基础产品销售信息在各材料中的定义没有冲突,[{'CLAUSE': '众安在线财产保险股份有限公司 附加旅行意外医疗费用补偿保险条款（互...
1,m_00002a,r_00002,该产品的赔付 & 领取规则在各材料中的定义没有冲突,[{'CLAUSE': '住院天数 指被保险人在医院实际的住院治疗日数。住院满 24 小时为...
2,m_00003a,r_00003,该产品的责任免除在各材料中的定义没有冲突,[{'ADDITIONAL_AGREEMENT': '内容填写表（放在第一个sheet） 层...
3,m_00004a,r_00004,该产品的投保条款在各材料中的定义没有冲突,[{'INSURE_NOTICE': '保单及发票 形式： 本合同采用电子保险单形式承保...
4,m_00005a,r_00005,该产品的投保条款在各材料中的定义没有冲突,[{'CLAUSE': '投保⼈、被保险⼈义务9. 保险期间和不保证续保7.'}]
...,...,...,...,...
162,m_00138a,r_00196,该产品的基础产品销售信息在各材料中的定义没有冲突,[{'CLAUSE': '所能提供的与确认保险事故的性质、原因等有关的其他证明和资料。如果委...
163,m_00120a,r_00197,该产品的附加条款在各材料中的定义没有冲突,[{'ADDITIONAL_AGREEMENT': '内容填写表（放在第一个sheet） 层...
164,m_00139a,r_00198,该产品的基础产品销售信息在各材料中的定义没有冲突,[{'CLAUSE': '释义⼀、分红保险⼆、周岁三、保单年度四、保险费约定⽀付⽇五、保单年...
165,m_00061a,r_00199,该产品的责任免除在各材料中的定义没有冲突,[{'ADDITIONAL_AGREEMENT': '本声明中，使用“本人”一词时，其含义包...


In [41]:
df_matched.to_csv('../data/finetune_extra_module.csv', index=False)

In [36]:
len(submit_cur_df[submit_cur_df['rule'].isna()])

33

In [32]:
submit_cur_df[['material_id','rule_id','result']].to_json('submit.jsonl', orient='records', lines=True)